In [75]:
# !pip install biopython
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [76]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio import pairwise2
import os
from tabulate import tabulate
import numpy as np

In [77]:
# Constant
label = ["Influenza A Virus", "Zaire Ebola Virus", "Other"]
maxLabel = 2
totalExperiment = 0

# Confusion matrix
confusionMatrix = [ ["Actual Values \ Predicted Values", label[0], label[1], label[2]],
                    [label[0], 0,0,0],
                    [label[1], 0,0,0],
                    [label[2], 0,0,0]]

trueMatrix = np.zeros([3,3])

# data path
influenzaPath = "./Bank/Influenza"
ebolaPath = "./Bank/Zaire Ebola Virus"

influenzaTestPath = "./Bank/Test/Influenza_Test.fasta"
ebolaTestPath = "./Bank/Test/ZaireEbola_Test.fasta"
otherTestPath = "./Bank/Test/Other_Test.fasta"

influenzaFileList = os.listdir(influenzaPath)
ebolaFileList = os.listdir(ebolaPath)

influenzaSeq = []
ebolaSeq = []
testSeq =[]

testSeqList = []

In [78]:
def fetchData(filePath, fileList):
    seqList = []
    for file in fileList:
        for sequence in SeqIO.parse(filePath + "/" + file,"fasta"):
            seqList.append(sequence)
    return seqList

In [79]:
def fetchTestData(filePath, expected):
    total = 0
    for sequence in SeqIO.parse(filePath,"fasta"):
        testSeqList.append([sequence, expected])
        total += 1
    return total

In [80]:
# fetch data
influenzaSeqList = fetchData(influenzaPath, influenzaFileList)
ebolaSeqList = fetchData(ebolaPath, ebolaFileList)

totalExperiment += fetchTestData(influenzaTestPath, label[0])
totalExperiment += fetchTestData(ebolaTestPath, label[1])
totalExperiment += fetchTestData(otherTestPath, label[2])

# for seq in testSeqList:
#     print(seq)


In [81]:
def localAlignment(testSeq, dataSeq):
    bestPercentage = 0
    bestSeqName = ""
    for seq_data in dataSeq:
        local_score = pairwise2.align.localms(testSeq.seq, seq_data.seq, 1, -1, -0.5, -0.1, score_only = True)
        local_percentage = (local_score / len(seq_data.seq)) * 100

        if local_percentage > bestPercentage:
            bestPercentage = local_percentage
            bestSeqName = seq_data.name
            if local_percentage == 100:
                return bestPercentage, bestSeqName
                
    return bestPercentage, bestSeqName

In [82]:
def getIdx(str):
    if str == label[0]:
        return 0
    elif str == label[1]:
        return 1
    else:
        return 2

In [83]:
def determineVerdict(percentage1, seqName1, percentage2, seqName2, expected):
    predicted = ""
    
    print(f"Virus name: {label[0]}")
    print(f"Best percentage: {percentage1}")
    print(f"Best sequence name: {seqName1}")
    print("")
    print(f"Virus name: {label[1]}")
    print(f"Best percentage: {percentage2}")
    print(f"Best sequence name: {seqName2}")
    print("")

    if percentage1 < 50 and percentage2 < 50:
        print("Both sequence are not quite convergent! It is either not match to both virus or please provide a more complete genome!")
        predicted = label[2]
    elif percentage1 == 100:
        print(f"Perfect match for {label[0]} is found! it is {seqName1} with percentage of {percentage1}!")
        predicted = label[0]
    elif percentage2 == 100:
        print(f"Perfect match for {label[1]} is found! it is {seqName2} with percentage of {percentage2}!")
        predicted = label[1]
    elif percentage1 > percentage2:
        print(f"Sequence alignment result of {label[0]} is better than {label[1]} with percentage of {percentage1}")
        predicted = label[0]
    else:
        print(f"Sequence alignment result of {label[1]} is better than {label[0]} with percentage of {percentage2}")
        predicted = label[1]

    print("\n\n\n-----------------------------------------\n")

    predictionIdx = getIdx(predicted)
    expectedIdx = getIdx(expected)

    confusionMatrix[expectedIdx+1][predictionIdx+1] += 1
    trueMatrix[expectedIdx][predictionIdx] += 1


In [84]:
for i, seq in enumerate(testSeqList):
    print(f"analyzing sequence: {seq[0].name}")
    print(f"Expected: {seq[1]}")
    bestInfluenzaPercentage, bestInfluenzaSeqName = localAlignment(seq[0], influenzaSeqList)
    bestEbolaPercentage, bestEbolaSeqName = localAlignment(seq[0], ebolaSeqList)

    print("Verdict:")
    determineVerdict(bestInfluenzaPercentage, bestInfluenzaSeqName, bestEbolaPercentage, bestEbolaSeqName, seq[1])


analyzing sequence: ON468446.1
Expected: Influenza A Virus
Verdict:
Virus name: Influenza A Virus
Best percentage: 72.96076099881084
Best sequence name: NC_026428.1

Virus name: Zaire Ebola Virus
Best percentage: 2.710058547391691
Best sequence name: NC_002549.1

Sequence alignment result of Influenza A Virus is better than Zaire Ebola Virus with percentage of 72.96076099881084



-----------------------------------------

analyzing sequence: ON468445.1
Expected: Influenza A Virus
Verdict:
Virus name: Influenza A Virus
Best percentage: 94.13441955193483
Best sequence name: NC_026431.1

Virus name: Zaire Ebola Virus
Best percentage: 3.1193628355925216
Best sequence name: NC_002549.1

Sequence alignment result of Influenza A Virus is better than Zaire Ebola Virus with percentage of 94.13441955193483



-----------------------------------------

analyzing sequence: ON468444.1
Expected: Influenza A Virus
Verdict:
Virus name: Influenza A Virus
Best percentage: 94.30861723446894
Best sequenc

In [85]:
table = tabulate(confusionMatrix, headers="firstrow", tablefmt="fancy_grid")
print("Confusion Matrix")
print(table)
confusionMatrix

Confusion Matrix
╒════════════════════════════════════╤═════════════════════╤═════════════════════╤═════════╕
│ Actual Values \ Predicted Values   │   Influenza A Virus │   Zaire Ebola Virus │   Other │
╞════════════════════════════════════╪═════════════════════╪═════════════════════╪═════════╡
│ Influenza A Virus                  │                  10 │                   0 │       0 │
├────────────────────────────────────┼─────────────────────┼─────────────────────┼─────────┤
│ Zaire Ebola Virus                  │                   0 │                  10 │       0 │
├────────────────────────────────────┼─────────────────────┼─────────────────────┼─────────┤
│ Other                              │                   3 │                   0 │       7 │
╘════════════════════════════════════╧═════════════════════╧═════════════════════╧═════════╛


[['Actual Values \\ Predicted Values',
  'Influenza A Virus',
  'Zaire Ebola Virus',
  'Other'],
 ['Influenza A Virus', 10, 0, 0],
 ['Zaire Ebola Virus', 0, 10, 0],
 ['Other', 3, 0, 7]]

In [93]:
def getAccuracy(mtx, idx):
    TP = mtx[idx, idx]

    temp = np.delete(np.arange(mtx.shape[0]), idx)
    TN = np.sum(mtx[temp[:, None], temp])

    FP = np.sum(mtx[:, idx]) - TP

    FN = np.sum(mtx[idx, :]) - TP
    
    print(f"True Positives: {int(TP)}")
    print(f"False Positives: {int(FP)}")
    print(f"True Negatives: {int(TN)}")
    print(f"False Negatives: {int(FN)}")

    # Accuracy
    Accuracy = (TP + TN) / totalExperiment
    Precision = (TP) / (TP + FP)
    Recall = (TP) / (TP + FN)
    F1 = 2 * ((Precision * Recall) / (Precision + Recall))

    print(f"Accuracy: {Accuracy}")
    print(f"Precision: {Precision}")
    print(f"Recall: {Recall}")
    print(f"F1: {F1}")

In [94]:
for i in range(0,3):
    print(f"Accuracy and precision for {label[i]}")
    getAccuracy(trueMatrix, i)

Accuracy and precision for Influenza A Virus
True Positives: 10
False Positives: 3
True Negatives: 17
False Negatives: 0
Accuracy: 0.9
Precision: 0.7692307692307693
Recall: 1.0
F1: 0.8695652173913044
Accuracy and precision for Zaire Ebola Virus
True Positives: 10
False Positives: 0
True Negatives: 20
False Negatives: 0
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1: 1.0
Accuracy and precision for Other
True Positives: 7
False Positives: 0
True Negatives: 20
False Negatives: 3
Accuracy: 0.9
Precision: 1.0
Recall: 0.7
F1: 0.8235294117647058
